## Where to Read More
This notebook will help me keep track of my experimental weighted crypto portfolio that I started for this year (2022). 

You can read more about the parameters and qualities of the portfolio in the [original blog post](https://andresberejnoi.com/best-cryptos-to-invest-in-2022/) I wrote. I write an update every month, which you can follow on my blog.

### Crypto Market Data
At the moment, I'm making use of Coinbase Pro API through the [cbpro library](https://github.com/danpaquin/coinbasepro-python) to collect market data of the coins I'm tracking. The endpoint I'll be using is `https://api.exchange.coinbase.com/products/{product_id}/candles`, and the documentation can be found [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

You can also use other APIs to gather the data. I was initially using AlphaVantage with an API key, but Coinbase Pro supported more of the coins I'm tracking. 
The endpoint I would use for AlphaVantage is `DIGITAL_CURRENCY_DAILY `, and the documentation can be found [here](https://www.alphavantage.co/documentation/#currency-daily). Additionally, [this csv](https://www.alphavantage.co/digital_currency_list/) file from AlphaVantage lists all the cryptocurrencies supported by the API.


In [1]:
import os
import pandas as pd
import requests
import time
import cbpro

#from api_keys import alpha_vantage_key   #if you use AlphaVantage, put your own key here or in a different file

##  Find File Containing Initial Purchase and Allocations

In [2]:
# Get the data for the portfolio
_initial_purchase_file = "2022 Weighted Crypto Portfolio.csv"
_data_folder           = "./data/"

data_path = os.path.join(_data_folder, _initial_purchase_file)
data_path

'./data/2022 Weighted Crypto Portfolio.csv'

## Weighted Portfolio
Below is the portfolio and percentage allocation for each coin as it was when I started it. It is good to have the starting point to compare to later.

In [3]:
df = pd.read_csv(data_path)
df

,Coin,Amount,Net Cost,Fees,Investment,Cost Basis,Exchange
0,BTC,0.000307,15.00,0.07,14.93,"48,623.69",Coinbase Pro
1,ETH,0.003787,15.00,0.07,14.93,"3,941.66",Coinbase Pro
2,BNB,0.023982,12.70,0.01,12.69,529.00,BinanceUS
3,SOL,0.058000,10.50,0.05,10.45,179.33,Coinbase Pro
4,ADA,7.120000,9.75,0.05,9.70,1.36,Coinbase Pro
5,WLUNA,0.104000,9.80,0.05,9.75,93.22,Coinbase Pro
6,AVAX,0.080000,9.79,0.05,9.74,121.33,Coinbase Pro
7,DOT,0.352000,9.80,0.05,9.75,27.68,Coinbase Pro
8,DOGE,42.300000,7.60,0.04,7.56,0.18,Coinbase Pro
9,SHIB,164.383000,6.00,0.03,5.97,0.00,Coinbase Pro


## Using Coinbase Pro Library to Get Historic Data
For the `get_historic_rates` function, the granularity for daily candles is 86400. Check the documentation [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

In [4]:
client = cbpro.PublicClient()

In [5]:
coinbase_data_individual_dfs = {}
quote_currency = 'USD'

for coin in df['Coin']:
    pair = f"{coin}-{quote_currency}"
    
    #cbpro will return a list of lists. We need to turn it into a dataframe
    columns = ['date','open','high','low','close','volume']
    nested_list_data = client.get_product_historic_rates(pair, start='2021-12-23', end='2022-02-28', granularity=86400)
    
    coinbase_data_individual_dfs[coin] = pd.DataFrame(nested_list_data, columns=columns).set_index('date',drop=True).sort_index(ascending=True)
    
    #convert date from unix time to iso format (easier to look at)
    coinbase_data_individual_dfs[coin].index = pd.to_datetime(
        coinbase_data_individual_dfs[coin].index,
        origin = 'unix',
        unit='s')

In [6]:
coinbase_data_individual_dfs['BTC']

,open,high,low,close,volume
date,,,,,
2021-12-23,48032.16,51397.82,48608.61,50842.20,16627.046280
2021-12-24,50445.55,51878.60,50842.06,50851.38,11690.454182
2021-12-25,50191.84,51171.68,50852.31,50428.31,5550.571532
2021-12-26,49460.00,51295.33,50428.31,50801.79,6863.394738
2021-12-27,50480.00,52100.00,50804.33,50717.77,11596.914682
...,...,...,...,...,...
2022-02-24,34322.00,39720.00,37268.18,38347.30,42124.518761
2022-02-25,38016.44,39716.00,38347.30,39238.48,20761.669245
2022-02-26,38592.04,40300.50,39242.71,39137.80,8601.671097


### Save Full Candle Files For Each Coin
Before saving, the index is sorted so that each candle is in ascending order (ascending towards the bottom).

In [7]:
daily_close_coins = {}   #store only the closing data. This will be turned into a dataframe later
for i, coin in enumerate(coinbase_data_individual_dfs):
    pair = f"{coin}_{quote_currency}"
    _output_file = f"{pair}_daily_candles.csv"
    output_file = os.path.join(_data_folder, _output_file)
    
    print(f"--> Saving candles for pair: {pair}\n\tat {output_file}\n")
    
    _df = coinbase_data_individual_dfs[coin]
    
    # collect closing data
    daily_close_coins[coin] = _df['close']
    
    # write dataframe to file
    _df.to_csv(output_file, 
               index=True,
               mode='a'  #python file mode. 'a' is for append, 'w' for write, etc
               )
    

--> Saving candles for pair: BTC_USD
	at ./data/BTC_USD_daily_candles.csv

--> Saving candles for pair: ETH_USD
	at ./data/ETH_USD_daily_candles.csv

--> Saving candles for pair: BNB_USD
	at ./data/BNB_USD_daily_candles.csv

--> Saving candles for pair: SOL_USD
	at ./data/SOL_USD_daily_candles.csv

--> Saving candles for pair: ADA_USD
	at ./data/ADA_USD_daily_candles.csv

--> Saving candles for pair: WLUNA_USD
	at ./data/WLUNA_USD_daily_candles.csv

--> Saving candles for pair: AVAX_USD
	at ./data/AVAX_USD_daily_candles.csv

--> Saving candles for pair: DOT_USD
	at ./data/DOT_USD_daily_candles.csv

--> Saving candles for pair: DOGE_USD
	at ./data/DOGE_USD_daily_candles.csv

--> Saving candles for pair: SHIB_USD
	at ./data/SHIB_USD_daily_candles.csv

--> Saving candles for pair: MATIC_USD
	at ./data/MATIC_USD_daily_candles.csv

--> Saving candles for pair: CRO_USD
	at ./data/CRO_USD_daily_candles.csv

--> Saving candles for pair: LTC_USD
	at ./data/LTC_USD_daily_candles.csv

--> Saving 

In [9]:
df_coinbase = pd.DataFrame(daily_close_coins)
df_coinbase

,BTC,ETH,BNB,SOL,ADA,WLUNA,AVAX,DOT,DOGE,SHIB,MATIC,CRO,LTC,UNI,LINK,ALGO,BCH,XLM,ATOM,AXS
date,,,,,,,,,,,,,,,,,,,,
2021-12-23,50842.20,4113.08,NaN,190.07,1.4743,94.42,121.37,29.17,0.1851,0.000039,2.6858,0.5951,163.74,18.18,22.19,1.4697,455.93,0.287776,28.38,107.47
2021-12-24,50851.38,4049.65,NaN,190.65,1.3935,95.77,114.93,28.17,0.1862,0.000036,2.4687,0.6280,161.35,17.50,21.47,1.5760,452.27,0.278628,26.87,104.62
2021-12-25,50428.31,4097.43,NaN,193.06,1.4542,98.39,114.86,28.86,0.1911,0.000038,2.6409,0.6434,158.00,17.49,22.12,1.5748,455.36,0.290047,29.79,109.27
2021-12-26,50801.79,4065.91,NaN,198.00,1.4549,99.63,115.07,31.34,0.1899,0.000038,2.8793,0.6233,155.96,18.67,22.99,1.6079,451.86,0.291987,32.17,107.62
2021-12-27,50717.77,4038.64,NaN,195.74,1.5164,90.88,113.96,30.97,0.1879,0.000039,2.7267,0.6234,155.78,19.11,23.02,1.6631,465.45,0.300288,29.68,107.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-24,38347.30,2597.42,NaN,89.41,0.8533,65.16,76.41,16.17,0.1242,0.000024,1.4414,0.3952,104.24,8.82,13.27,0.7779,300.96,0.182403,24.86,48.82
2022-02-25,39238.48,2769.52,NaN,92.84,0.8997,73.07,81.05,17.28,0.1277,0.000025,1.5525,0.4183,109.33,9.25,14.39,0.8312,309.04,0.191142,27.30,51.16
2022-02-26,39137.80,2780.71,NaN,90.32,0.8873,78.13,81.79,18.04,0.1277,0.000024,1.5161,0.4142,108.29,9.06,14.72,0.8598,315.56,0.192899,29.89,50.44


### Save Single File With Only Closing Data For All Coins Combined

In [10]:
close_candle_output_file = os.path.join(_data_folder, "2022_portfolio_daily_close.csv")
df_coinbase.to_csv(close_candle_output_file,
                   index=True,
                   mode='a')

---
---

## Retrieving Historic Data With API From ALPHA VANTAGE
The AlphaVantage section below is going to be turned into markdown so I don't accidentally run it. If you want to use it, simply turn those cells into code cells.

In [4]:
def get_daily_data(symbol, api_key, start_date=None, end_date=None, quote_currency='USD', simplify=True):
    '''Uses AlphaVantage endpoint DIGITAL_CURRENCY_DAILY to return a dataframe.
    I created this function to simplify the process'''
    
    _base_url    = "https://www.alphavantage.co/"
    api_function = 'DIGITAL_CURRENCY_DAILY'

    url = f'{_base_url}query?function={api_function}&symbol={symbol}&market={quote_currency}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    
    #check that we received proper data
    if 'Error Message' in data:
        print(f"\t--> Error retrieving {symbol}. It's probably not supported.")
        print("\t    Returning `None`")
        return None
    
    #we need to parse data.
    _candle_data = data["Time Series (Digital Currency Daily)"]
    df = pd.DataFrame(_candle_data).transpose()  #without transpose, we get each daily candle as a column. We want each candle as a row, and each column as the OHLC data
    
    #--convert index to datetime
    df.index = pd.to_datetime(df.index)
    
    #AlphaVantage returns data both in USD and whatever quote currency we asked for. 
    # in case we do not want repetition, simplify will remove the fixed USD market and return a clearner dataframe just with the requested quote currency
    if simplify:
        desired_columns = ['1a. open (USD)','2a. high (USD)','3a. low (USD)','4a. close (USD)','5. volume','6. market cap (USD)']
        new_names       = ['open','high','low','close','volume','market_cap']
        
        dict_replacement = {old_name:new_name for old_name, new_name in zip(desired_columns, new_names)}
        df = df[desired_columns]
        
        df = df.rename(columns=dict_replacement)
    
    #-- in case start and end dates are specified:
    if start_date:
        if end_date:
            return df[start_date:end_date]
        else:
            return df[start_date:]
    elif end_date:
        return df[:end_date]
    
    return df

start_date = "2022-01-01"
end_date   = "2022-02-28"

daily_close_candles = {}
problematic_coins = []  #used to collect coins that could not be found in AlphaVantage API

for coin in df['Coin']:
    print(f"--> Parsing data for {coin}")
    
    if coin in daily_close_candles:  #skip coins that are already in the candle dict
        continue
        
    if coin=='WLUNA':
        coin = 'LUNA'  #alphavantage does not support WLUNA, so we used the real token
        print("\t* Changed WLUNA to LUNA")
    candles = get_daily_data(symbol         = coin,
                             api_key        = alpha_vantage_key,
                             start_date     = start_date,
                             end_date       = end_date,
                             quote_currency = 'USD',
                             simplify       = True)
    
    if candles is None:
        problematic_coins.append(coin)
        continue
        
    close_candles = candles['close']  # I only want the closing data
    daily_close_candles[coin] = close_candles
    
    time.sleep(13)  #AlphaVantage allows 5 requests per minute, so I'm delaying between requests

portfolio_df = pd.DataFrame(daily_close_candles)
portfolio_df

output_file = os.path.join(_data_folder, "portfolio_daily_close.csv")
portfolio_df.to_csv(output_file)